# Programación en entornos BigData. Entrega 4

* En todos los gráficos se valorará que se pueda entender fácilmente la información mostrada (etiquetas, leyendas, título).
* Cada ejercicio vale 5 puntos, 2.5 puntos cada apartado.
* También se valorará la calidad del código, incluyendo comentarios, y del resto del notebook.
* En todos los ejercicios hay que usar Plotly, ipywidgets o Dash.
* La entrega se realizará a través del aula virtual.
* Se entregará un único notebook con nombre nombre_apellidos_entrega4_.ipynb.



In [ ]:
# $ pip install fiona==1.8.20

In [ ]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

import xml.etree.ElementTree as ET
import requests
import plotly.express as px
import plotly.io as pio

from ipywidgets import interact, widgets

import geopandas as gpd
import json

In [ ]:
# para google colab
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Clara del Rey/BIG DATA/Programacion/FTP/000 Datos/"

In [ ]:
# Local: VSCode
# path = "./"

1.

a) Crea un mapa con los terremotos ocurridos en los diez últimos días a partir de  la información del IGN.
http://www.ign.es/ign/RssTools/sismologia.xml

En cada punto se mostrarán fecha, magnitud, longitud y latitud al pasar el ratón por encima.

El color y el tamaño del punto tienen que depender de la magnitud.

(2.5 puntos)




Introduce tu solución a partir de aquí...




In [ ]:
# Descargar y parsear el archivo XML
url = "https://www.ign.es/ign/RssTools/sismologia.xml"
response = requests.get(url)
root = ET.fromstring(response.content)

In [ ]:
# Seleccionamos los terremotos
terremotos = root.findall(".//item")

In [ ]:
for terremoto in terremotos[:5]:
    desc = terremoto.find(".description").text
    print(desc)

In [ ]:
terremotos_10 = []

for terremoto in terremotos:
    desc = terremoto.find("description").text

    # Extraer fecha, magnitud, longitud y latitud, del texto:
    try:
        # Fecha
        fecha_str = desc.split("fecha")[1].split(" ")[1].strip()
        fecha = dt.strptime(fecha_str, "%d/%m/%Y").date()

        # Magnitud
        magnitud_str = desc.split("magnitud")[1].split(" ")[1].strip()
        magnitud = float(magnitud_str)

        # Coordenadas
        coords = desc.split("localización:")[1].strip()

        coordenadas = coords.split(",")
        latitud = float(coordenadas[0])
        longitud = float(coordenadas[1])

        # print(f"Fecha: {fecha}, Magnitud: {magnitud}, Latitud: {latitud}, Longitud: {longitud}")

        # Añadir solo terremotos de los últimos 10
        if dt.now().date() - fecha <= timedelta(days=10):
            terremotos_10.append({
                "Fecha": fecha,
                "Magnitud": magnitud,
                "Latitud": latitud,
                "Longitud": longitud,
            })

    except Exception as e:
        print(f"Error al procesar: {desc}")
        print(f"Detalle del error: {e}")

In [ ]:
for terremoto in terremotos_10[:5]:
    print(terremoto)

In [ ]:
df_terremotos_10 = pd.DataFrame(terremotos_10)
df_terremotos_10.head()

In [ ]:
# Crear el mapa interactivo con Plotly
fig = px.scatter_geo(
    df_terremotos_10,
    lat="Latitud",
    lon="Longitud",
    color="Magnitud",
    size="Magnitud",
    hover_data=["Fecha", "Magnitud", "Latitud", "Longitud"],
    title="Terremotos en los Últimos 10 días",
    color_continuous_scale="Viridis",
)

# Mostrar el mapa interactivo
fig.show()

b) Usando ipywidgets (o Dash), crea un mapa como el anterior pero con controles interactivos para:
* Mostrar solo terremotos en cierto rango de magnitud.
* Cambiar el número de días (entre 1 y 10)

(2.5 puntos)


Introduce tu solución a partir de aquí...


In [ ]:
#################################
### CONSULTAR CON EL PROFESOR ###
#################################

# NO funciona: Colab
# SI funciona: VSCode
# NO consigo que se actualice el gráfico (refresh), sólo que aparezcan nuevos gráficos debajo

In [ ]:
# Configurar el notebook para colab
pio.renderers.default = 'notebook'

In [ ]:
from ipywidgets import interact, widgets, Output
import plotly.express as px
from datetime import datetime as dt, timedelta

# Contenedor de salida
output = Output()

@interact(
    dias=widgets.IntSlider(value=10, min=1, max=10, step=1, description='Días:', continuous_update=False),
    magnitud_min=widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='Mag. Min:', continuous_update=False),
    magnitud_max=widgets.FloatSlider(value=10, min=0, max=10, step=0.1, description='Mag. Max:', continuous_update=False),
)
def generar_mapa(dias=10, magnitud_min=0, magnitud_max=10):
    with output:
        output.clear_output(wait=True)  # Limpia gráficos anteriores
        fecha_limite = dt.now().date() - timedelta(days=dias)
        df_filtrado = df_terremotos_10[
            (df_terremotos_10["Fecha"] >= fecha_limite) &
            (df_terremotos_10["Magnitud"] >= magnitud_min) &
            (df_terremotos_10["Magnitud"] <= magnitud_max)
        ]

        # Crear el mapa interactivo
        fig = px.scatter_geo(
            df_filtrado,
            lat="Latitud",
            lon="Longitud",
            color="Magnitud",
            size="Magnitud",
            hover_data=["Fecha", "Magnitud", "Latitud", "Longitud"],
            title=f"Terremotos de los últimos {dias} días (Magnitud: {magnitud_min}-{magnitud_max})",
            color_continuous_scale="Viridis",
        )

        fig.show()

display(output)


2.

a) Crea un mapa coroplético que muestre las salidas realizadas por los bomberos en cada distrito de Madrid. Utiliza las salidas totales para el año 2023.

Formas de los distritos:

 https://gestiona.comunidad.madrid/nomecalles/DescargaBDTCorte.icm


Datos de bomberos


https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fa677996afc6f510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default


(2.5 puntos)


Introduce tu solución a partir de aquí...


Los distritos de Madrid se han obtenido de aquí:

https://geoportal.madrid.es/IDEAM_WBGEOPORTAL/dataset.iam?id=541f4ef6-762b-11e9-861d-ecb1d753f6e8

In [ ]:
# Descargar y cargar las formas de los distritos
distritos = gpd.read_file(path + "distritos_madrid/Distritos.shp")

# Verificar las primeras filas
distritos.head()

In [ ]:
# Agrupar las salidas totales por distrito
distritos = distritos[["DISTRI_MAY", "geometry"]]
distritos.head()

In [ ]:
# Descargar y cargar los datos de las salidas de bomberos
bomberos_2023 = pd.read_csv(path + "ActuacionesBomberos_2023.csv", sep=";")

# Verificar las primeras filas
bomberos_2023.head()

In [ ]:
# Agrupar las salidas totales por distrito
salidas_x_distrito = bomberos_2023.groupby("DISTRITO")["TOTAL"].sum().reset_index()

# Verificar los datos agrupados
salidas_x_distrito.head()

In [ ]:
# Vamos a utilizar este:
distritos['DISTRI_MAY'].unique()

In [ ]:
salidas_x_distrito['DISTRITO'].unique()

In [ ]:
# Normalizar los datos:
# Diccionario de para normalizar los distritos
correcciones = {
    "FUENCARRAL - EL PARDO": "FUENCARRAL",
    "MONCLOA - ARAVACA": "MONCLOA",
    "PUENTE DE VALLECAS": "PUENTE VALLECAS",
    "SAN BLAS - CANILLEJAS": "SAN BLAS",
    "VICÁLVARO": "VICALVARO"
}

# Aplicar las correcciones
distritos["DISTRI_MAY"] = distritos["DISTRI_MAY"].replace(correcciones)

In [ ]:
# Filtrar distritos irrelevantes
salidas_x_distrito = salidas_x_distrito[
    ~salidas_x_distrito["DISTRITO"].str.contains("FUERA")
]

In [ ]:
# Unir datos geográficos con salidas
distritos_merged = distritos.merge(salidas_x_distrito, left_on="DISTRI_MAY", right_on="DISTRITO")

# Verificar el resultado
distritos_merged

In [ ]:
# Garantizamos la compatibilidad de los datos:

# Convertir a EPSG:4326 (WGS84) --> World Geodesic System 1984
distritos_merged = distritos_merged.to_crs(epsg=4326)

# Convertir a GeoJSON
geojson_data = json.loads(distritos_merged.to_json())

print(geojson_data["features"][0])

In [ ]:
# Crear el mapa coroplético
fig = px.choropleth_mapbox(
    distritos_merged,

    # Geometria de los distritos
    geojson=geojson_data,

    # Identifica cada distrito usando el indice
    locations=distritos_merged.index,

    color="TOTAL",
    hover_name="DISTRI_MAY",
    title="Salidas de Bomberos por Distrito (2023)",

    # https://carto.com/blog/getting-to-know-positron-and-dark-matter
    mapbox_style="carto-positron",
    hover_data={"TOTAL": True},

    color_continuous_scale="YlOrRd", # Yellow Orange Red
    center={"lat": 40.416775, "lon": -3.703790},  # Coordenadas del centro de Madrid
    zoom=9
)

# Mostrar el mapa
fig.show()

b) Modifica el mapa anterior de la siguiente manera:
* Añade un control para seleccionar el tipo de salida, de entre los presentes en el fichero.
* Utilizando también los datos del año 2022, haz una versión animada.

(2.5 puntos)

Introduce tu solución a partir de aquí...


In [ ]:
# Descargar y cargar los datos de las salidas de bomberos
bomberos_2022 = pd.read_csv(path + "ActuacionesBomberos_2022.csv", sep=";")

# Verificar las primeras filas
bomberos_2022.head()

In [ ]:
# Combinar ambos años y agregar un campo de año
bomberos = pd.concat([
    bomberos_2022.assign(AÑO=2022),
    bomberos_2023.assign(AÑO=2023)
])

bomberos.head()

In [ ]:
# Agrupar datos por año, mes, distrito y tipo de salida
bomberos_melted = bomberos.melt(
    id_vars=["AÑO", "MES", "DISTRITO"],
    value_vars=[
        "FUEGOS",
        "DAÑOS EN CONSTRUCCION",
        "SALVAMENTOS Y RESCATES",
        "DAÑOS POR AGUA",
        "INCIDENTES DIVERSOS",
        "SALIDAS SIN INTERVENCION",
        "SERVICIOS VARIOS",
        "TOTAL"
    ],
    var_name="TIPO_SALIDA",
    value_name="SALIDAS"
)

bomberos_melted.head()

In [ ]:
# Los distritos
distritos.head()

In [ ]:
# Unir los datos geográficos con las salidas de bomberos
distritos_merged = distritos.merge(
    bomberos_melted,
    left_on="DISTRI_MAY",
    right_on="DISTRITO"
)

distritos_merged.head()

In [ ]:
# Convertir MES a un formato numérico
meses_orden = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

distritos_merged["MES"] = distritos_merged["MES"].map(meses_orden)

# Ordenar por MES (y opcionalmente AÑO)
distritos_merged = distritos_merged.sort_values(by=["AÑO", "MES"], ascending=[True, True])

# Cambiar los meses y ponerlos de 1-24
distritos_merged["MES_EXT"] = (distritos_merged["AÑO"] - 2022) * 12 + distritos_merged["MES"]

distritos_merged.head()

In [ ]:
# Garantizamos la compatibilidad de los datos:

# Convertir a EPSG:4326 (WGS84) --> World Geodesic System 1984
distritos_merged = distritos_merged.to_crs(epsg=4326)

# Simplificar las geometrías para reducir la complejidad: animacion mas rapida
# Ajusta la tolerancia según sea necesario
distritos_merged["geometry"] = distritos_merged["geometry"].simplify(tolerance=0.005)

# Convertir a GeoJSON
geojson_data = json.loads(distritos_merged[["geometry"]].to_json())

print(geojson_data["features"][0])

In [ ]:
# Función para mostrar el mapa con animación mensual
@interact(
    tipo_salida=widgets.Dropdown(
        options=distritos_merged["TIPO_SALIDA"].unique(),
        value="FUEGOS",
        description="Tipo de Salida:"
    )
)
def mostrar_mapa(tipo_salida):
    # Filtrar por tipo de salida
    datos_filtrados = distritos_merged[distritos_merged["TIPO_SALIDA"] == tipo_salida]

    # Excluir valores nulos o cero
    datos_filtrados = datos_filtrados.dropna(subset=["SALIDAS"])
    datos_filtrados = datos_filtrados[datos_filtrados["SALIDAS"] > 0]

    # Crear el mapa coroplético con animación mensual
    fig = px.choropleth_mapbox(
        datos_filtrados,
        geojson=geojson_data,
        locations=datos_filtrados.index,
        color="SALIDAS",
        hover_name="DISTRI_MAY",
        hover_data={"SALIDAS": True, "AÑO": True, "MES": True},
        animation_frame="MES_EXT",  # Animación por mes numérico
        color_continuous_scale="YlOrRd",

        # Rango fijo para la escala de colores
        range_color=[0, datos_filtrados["SALIDAS"].max()],
        # range_color=[0, 50],

        mapbox_style="carto-positron",
        title=f"Tipo de Salida 2022-2023: {tipo_salida}",
        center={"lat": 40.416775, "lon": -3.703790},
        zoom=9
    )

    # Duración de la transición en milisegundos
    fig.update_layout(transition={'duration': 1000})

    # Mostrar el mapa
    fig.show()

Sin la animación: la suma para todas las salidas de 2023

In [ ]:
distritos_merged.head()

In [ ]:
distritos_agrupados = distritos_merged.groupby(
    ["AÑO", "DISTRI_MAY", "geometry", "TIPO_SALIDA"]
)["SALIDAS"].sum().reset_index()

distritos_agrupados.head()

In [ ]:
# Filtrar datos del año 2023
distritos_2023 = distritos_agrupados[distritos_agrupados["AÑO"] == 2023]
distritos_2023.head()

In [ ]:
# Garantizamos la compatibilidad de los datos:

# Convertir distritos_agrupados en un GeoDataFrame
distritos_agrupados = gpd.GeoDataFrame(
    distritos_agrupados,
    geometry="geometry"
)

# Convertir a EPSG:4326 (WGS84) --> World Geodesic System 1984
distritos_agrupados = distritos_agrupados.to_crs(epsg=4326)

# Simplificar las geometrías para reducir la complejidad: animacion mas rapida
# Ajusta la tolerancia según sea necesario
distritos_agrupados["geometry"] = distritos_agrupados["geometry"].simplify(tolerance=0.0001)

# Convertir a GeoJSON
geojson_data = json.loads(distritos_agrupados[["geometry"]].to_json())

print(geojson_data["features"][0])

In [ ]:
# Función para mostrar el mapa filtrado por tipo de salida
@interact(
    tipo_salida=widgets.Dropdown(
        options=distritos_2023["TIPO_SALIDA"].unique(),  # Tipos de salida disponibles
        value="TOTAL",  # Valor por defecto
        description="Tipo de Salida:"
    )
)
def mostrar_mapa_estatico(tipo_salida):
    # Filtrar los datos según el tipo de salida seleccionado
    datos_filtrados = distritos_2023[distritos_2023["TIPO_SALIDA"] == tipo_salida]

    # Crear el mapa coroplético
    fig = px.choropleth_mapbox(
        datos_filtrados,

        # Geometria de los distritos
        geojson=geojson_data,

        # Identifica cada distrito usando el índice
        locations=datos_filtrados.index,

        color="SALIDAS",  # Columna que se usará para los colores
        hover_name="DISTRI_MAY",
        title=f"Salidas de Bomberos por Distrito (2023): {tipo_salida}",

        # Mapa base y datos adicionales
        mapbox_style="carto-positron",
        hover_data={"SALIDAS": True},

        # Configuración del color
        color_continuous_scale="YlOrRd",
        range_color=[0, datos_filtrados["SALIDAS"].max()],  # Ajustar rango dinámico de colores

        # Centro del mapa
        center={"lat": 40.416775, "lon": -3.703790},
        zoom=9
    )

    # Mostrar el mapa
    fig.show()
